这个文件用于尝试通过图片和mask 生成 点云 和 标定数据。brownv1的前提下，更换这两个数据集即可 获得brownv2数据集

该版本增加了参数方式，可以通过修改参数使得调用matlab生成不一样的东西

In [ ]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
 #该文件的目标是，单纯的修改人物编号就可以生成点云，然后利用点云生成对应的patch
#  即全部的文件结构都如下面所示，只需遍历人物编号即可


In [1]:
import os
import shutil
def copy_frame(source_root, destination_path, num_frames):
    """Copies the first frame of each camera to the destination path."""
    os.makedirs(destination_path, exist_ok=True)
    i = 0
    dir1 = source_root
    for folder_name in sorted(os.listdir(dir1)):
        if folder_name.startswith("cam"):
            dir2 = os.path.join(dir1, folder_name)
            for file_name in os.listdir(dir2):
                if file_name.startswith(f"frame_{str(num_frames).zfill(5)}"):
                    i += 1
                    src_path = os.path.join(dir2, file_name)
                    if 'mask' in source_root:
                        dst_path = os.path.join(destination_path, f"{i}_mask.png")
                    else:
                        dst_path = os.path.join(destination_path, f"{i}.png")
                    shutil.copyfile(src_path, dst_path)
                    break # Only copy the first matching frame per camera folder
    print("images和mask 移动完毕")

In [ ]:
# 传入参数 执行psift 提取
# 调用该matlab文件来生成psift


# 输入文件的 组成结构:
#  -data_path(可以是任意路径，下面的路径按着相同的结构来就行，只有人物编号可以修改)
#  ++人物编号===
#  +++EMO-1-shout+laugh
#+ ++++10
#  +++++psiftproject
#  ++++++images
#  ++++++mask
#  ++++++descriptors
#  ++++++keypoints



# 该block 是将以前brown v1保存下来的数据集 修改成用于生成brown v2的格式 
import os
import shutil
human_number ='056'



# 修改文件夹使得文件夹结构存在project 
# 复制好图片和mask到project中
human_full_path  =  os.path.join('/media/DGST_data/Test_Data',str(human_number),'EMO-1-shout+laugh','1')
project_path = os.path.join(human_full_path,"psiftproject")
project_images_path= os.path.join(project_path,"images")
project_mask_path= os.path.join(project_path,"mask")

# 移动图片
source_images_path = os.path.join("/media/DGST_data/Data", human_number)
source_mask_path = os.path.join("/media/DGST_data/Data", human_number,'mask')

copy_frame(source_images_path, project_images_path, 1)
copy_frame(source_mask_path, project_mask_path, 1)


images和mask 移动完毕
images和mask 移动完毕


In [5]:
# 传入参数 执行psift 提取
# 调用该matlab文件来生成psift

# 输入参数为： 人物编号  所需kpts
import subprocess
import os
get_sift_m='/media/Trajectory3D/tools/get_sift/parameter_eth_get_face_sift1.m'

#人物编号
# 定义参数
dataset_path_call_matlab = '/media/DGST_data/Test_Data'
get_sift_human_number =  human_number
kpts_number = 10000

# 将参数写入 txt 文件
with open("/media/Trajectory3D/tools/ply/get_sift_parameters.txt", "w") as f:
    f.write(f"dataset_path_call_matlab={dataset_path_call_matlab}\n")
    f.write(f"human_number={get_sift_human_number}\n")
    f.write(f"kpts_number={kpts_number}\n")

print("参数已写入 parameters.txt")

#调用matlab
subprocess.run(["/usr/local/Matlab/R2020a/bin//matlab", "-nodisplay", "-nosplash", "-r", "run('{}'); exit".format(get_sift_m)])

参数已写入 parameters.txt

                            < M A T L A B (R) >
                  Copyright 1984-2020 The MathWorks, Inc.
                  R2020a (9.8.0.1323502) 64-bit (glnxa64)
                             February 25, 2020

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
dataset_path: /media/DGST_data/Test_Data
human_number: 056
kpts_number: 10000
   666


people =

  1x1 cell array

    {'056'}


people_num =

    '056'


scale =

     2


ans =

        3208        2200


ans =

        3208        2200

PSIFT: computing scale space...(19.694 s gss; PSIFT scale space parameters [PropertyName in brackets]
  sigman [SigmaN]        : 0.500000
  sigma0 [Sigma0]        : 1.090508
       O [NumOctaves]    : 3
       S [NumLevels]     : 8
    omin [FirstOctave]   : 0
    smin                 : -1
    smax                 : 8

outputpath =

    'Afunc_detect_pore_no1.png'

PSIFT detector parameters
  thersh [Threshold]     : 4.329462e-02
       r [Ed

CompletedProcess(args=['/usr/local/Matlab/R2020a/bin//matlab', '-nodisplay', '-nosplash', '-r', "run('/media/Trajectory3D/tools/get_sift/parameter_eth_get_face_sift1.m'); exit"], returncode=0)

In [7]:
#挖取patch并保存   
import subprocess
# 调用该文件 进行patch的挖取，此处挖取所得的patch大小为  (patch_size/2-1)*2+1 ,可以根据所需patch1大小进行修改patch_size的参数
patch_size = 32
patch_folder_path = os.path.join(dataset_path_call_matlab,human_number,'EMO-1-shout+laugh/10/psiftproject/images')
patch_folder_path_pathkp = os.path.join(dataset_path_call_matlab,human_number,'EMO-1-shout+laugh/10/psiftproject/keypoints')
patch_save_path = os.path.join(dataset_path_call_matlab,human_number,'EMO-1-shout+laugh/10/psiftproject/descriptors')
# 确保patch_save_path路径存在
if not os.path.exists(patch_save_path):
    os.makedirs(patch_save_path)


with open("/media/Trajectory3D/tools/ply/get_patch_parameters.txt", "w") as f:
    f.write(f"patch_size={patch_size}\n")
    f.write(f"patch_folder_path={patch_folder_path}\n")
    f.write(f"patch_folder_path_pathkp={patch_folder_path_pathkp}\n")
    f.write(f"patch_save_path={patch_save_path}\n")

extract_patch_m='/media/Trajectory3D/tools/ply/local-feature-evaluation-master/scripts/parameter_extract_patch.m'
subprocess.run(["/usr/local/Matlab/R2020a/bin//matlab", "-nodisplay", "-nosplash", "-r", "run('{}'); exit".format(extract_patch_m)])


                            < M A T L A B (R) >
                  Copyright 1984-2020 The MathWorks, Inc.
                  R2020a (9.8.0.1323502) 64-bit (glnxa64)
                             February 25, 2020

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
py_patch_size:  
folder_path: /media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/images
folder_path_pathkp: /media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/keypoints
save_path: /media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/1.png.bin.patches.mat
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/10.png.bin.patches.mat
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/11.png.bin.patches.mat
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/12.png.bin.patches.mat
/media/DGST_data/Test_D

CompletedProcess(args=['/usr/local/Matlab/R2020a/bin//matlab', '-nodisplay', '-nosplash', '-r', "run('/media/Trajectory3D/tools/ply/local-feature-evaluation-master/scripts/parameter_extract_patch.m'); exit"], returncode=0)

In [8]:
#提取出描述子并保存
#调用python   feature_extraction_hynet_input32.py
# 加载hynet模型
from brownv2_hymodel import HyNet
import torch
gpu_id=0
device = torch.device("cuda:{}".format(gpu_id) if torch.cuda.is_available() else "cpu")
net_modelpath = '/media/Trajectory3D/tools/ply/net-best.pth'
model = HyNet().eval()
model.to(device)
model.load_state_dict(torch.load(net_modelpath))


<All keys matched successfully>

In [10]:
#  可以根据需要将输入进来的64patch resize成32再输入到网络中，此处由于前面提取的是31大小的patch
#  故而不需要resize，如果前面的挖取patch1的大小修改，则需要将输入resize成32*32
import time
import scipy.io
import numpy as np
def write_matrix(path, matrix):
    with open(path, "wb") as fid:
        shape = np.array(matrix.shape, dtype=np.int32)
        shape.tofile(fid)
        matrix.tofile(fid)

def read_mat_file(file_path):
    # 读取 .mat 文件
    mat_contents = scipy.io.loadmat(file_path)
    
    # 返回读取的内容
    return mat_contents


#读入patch数据
batch_size = 512
image_names = os.listdir(patch_folder_path)
for i, image_name in enumerate(image_names):

    patches_path = os.path.join(patch_save_path,
                                image_name + ".bin.patches.mat")
    print(patches_path)
    if not os.path.exists(patches_path):
        continue

    print("Computing features for {} [{}/{}]".format(
            image_name, i + 1, len(image_names)), end="")

    start_time = time.time()

    descriptors_path = os.path.join(patch_save_path,
                                    image_name + ".bin")
    # if os.path.exists(descriptors_path):
    #     print(" -> skipping, already exist")
    #     continue
    mat_data = read_mat_file(patches_path)
    patches31 = np.array(mat_data["patch"])

    if patches31.ndim != 3:
        print(" -> skipping, invalid input")
        write_matrix(descriptors_path, np.zeros((0, 512), dtype=np.float32))
        continue

    patches = np.empty((patches31.shape[0], 32, 32), dtype=np.float32)
    patches[:, :31, :31] = patches31
    patches[:, 31, :31] = patches31[:, 30, :]
    patches[:, :31, 31] = patches31[:, :, 30]
    patches[:, 31, 31] = patches31[:, 30, 30]

    descriptors = []
    for i in range(0, patches.shape[0], batch_size):
        patches_batch = \
            patches[i:min(i + batch_size, patches.shape[0])]
        patches_batch = \
            torch.from_numpy(patches_batch[:, None]).float().to(device)
        # descriptors.append(tfeat(patches_batch).detach().cpu().numpy())
        descriptors.append(model(patches_batch).detach().cpu().numpy())

    if len(descriptors) == 0:
        descriptors = np.zeros((0, 128), dtype=np.float32)
    else:
        descriptors = np.concatenate(descriptors)

    write_matrix(descriptors_path, descriptors)

    print(" in {:.3f}s".format(time.time() - start_time))

#生成描述子并保存

/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/7.png.bin.patches.mat
Computing features for 7.png [1/16] in 2.181s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/1.png.bin.patches.mat
Computing features for 1.png [2/16] in 0.645s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/5.png.bin.patches.mat
Computing features for 5.png [3/16] in 0.318s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/8.png.bin.patches.mat
Computing features for 8.png [4/16] in 0.329s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/3.png.bin.patches.mat
Computing features for 3.png [5/16] in 0.323s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/6.png.bin.patches.mat
Computing features for 6.png [6/16] in 0.307s
/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject/descriptors/9.png.bin.patches.mat
Computing features for 9.png [7

In [11]:
#生成matches
import subprocess
# 输入参数，用于生成matches
# dataset_root 人物编号  前面的路径，可自定义
# human_number 制作哪一个number
dataset_root = dataset_path_call_matlab
match_human_number = human_number
with open("/media/Trajectory3D/tools/ply/matching_parameters.txt", "w") as f:
    f.write(f"dataset_root={dataset_root}\n")
    f.write(f"match_human_number={match_human_number}\n")
#创建matching 文件夹
match_m='/media/Trajectory3D/tools/ply/local-feature-evaluation-master/scripts/parameter_matching_pipeline.m'
subprocess.run(["/usr/local/Matlab/R2020a/bin//matlab", "-nodisplay", "-nosplash", "-r", "run('{}'); exit".format(match_m)])


                            < M A T L A B (R) >
                  Copyright 1984-2020 The MathWorks, Inc.
                  R2020a (9.8.0.1323502) 64-bit (glnxa64)
                             February 25, 2020

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
DATASET_ROOT: /media/DGST_data/Test_Data
    {'056'}

/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/10/psiftproject
Matching block [1/1, 1/1]Warning: The CUDA driver must recompile the GPU libraries because your device
is more recent than the libraries. Recompiling can take several minutes. 
> In exhaustive_matching (line 37)
  In parameter_matching_pipeline (line 132)
  In run (line 91) 
 in 447.017s
matches的总数是: 489644


CompletedProcess(args=['/usr/local/Matlab/R2020a/bin//matlab', '-nodisplay', '-nosplash', '-r', "run('/media/Trajectory3D/tools/ply/local-feature-evaluation-master/scripts/parameter_matching_pipeline.m'); exit"], returncode=0)

In [2]:
#跑后面的程序，生成ply和稀疏
#复制colmap所需的db文件，这其实只需要有一个正确格式的db就ok，里面存储什么数据都无所谓，因为会删除，重新写入

import os
import shutil
human_number ='056'


# 修改文件夹使得文件夹结构存在project 
# 复制好图片和mask到project中
human_full_path  =  os.path.join('/media/DGST_data/Test_Data',str(human_number),'EMO-1-shout+laugh','18')
project_path = os.path.join(human_full_path,"psiftproject")



import shutil
format_db_path = '/media/Trajectory3D/tools/ply/origin_format_database.db'
reconstruct_dataset_path = project_path
shutil.copy(format_db_path, os.path.join(reconstruct_dataset_path,'database.db'))
# 跑 reconstruction
import subprocess
reconstruct_py = '/media/Trajectory3D/tools/ply/local-feature-evaluation-master/scripts/parameter_reconstruction_pipeline.py'

args = ['python', reconstruct_py, f'--dataset_path={reconstruct_dataset_path}']
# 使用 subprocess.run 执行脚本并传递参数
subprocess.run(args)



/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/18/psiftproject
Importing features for 1.png
Importing features for 10.png
Importing features for 11.png
Importing features for 12.png
Importing features for 13.png
Importing features for 14.png
Importing features for 15.png
Importing features for 16.png
Importing features for 2.png
Importing features for 3.png
Importing features for 4.png
Importing features for 5.png
Importing features for 6.png
Importing features for 7.png
Importing features for 8.png
Importing features for 9.png
Importing matches for 10.png --- 7.png
Importing matches for 3.png --- 7.png
Importing matches for 13.png --- 8.png
Importing matches for 11.png --- 15.png
Importing matches for 13.png --- 16.png
Importing matches for 10.png --- 11.png
Importing matches for 10.png --- 14.png
Importing matches for 3.png --- 6.png
Importing matches for 2.png --- 4.png
Importing matches for 10.png --- 8.png
Importing matches for 14.png --- 9.png
Importing matches for 14.png --- 4

I0216 06:37:09.584954 26712 misc.cc:198] 
Custom feature matching
I0216 06:37:10.030508 26712 feature_matching.cc:1021] Matching block [1/1]
I0216 06:37:15.408103 26712 feature_matching.cc:46]  in 5.378s
I0216 06:37:15.430642 26712 timer.cc:91] Elapsed time: 0.097 [minutes]


/media/DGST_data/Test_Data/056/EMO-1-shout+laugh/18/psiftproject/sparse


I0216 06:37:15.659313 26802 misc.cc:198] 
Loading database
I0216 06:37:15.660429 26802 database_cache.cc:54] Loading cameras...
I0216 06:37:15.660466 26802 database_cache.cc:64]  16 in 0.000s
I0216 06:37:15.660471 26802 database_cache.cc:72] Loading matches...
I0216 06:37:15.662395 26802 database_cache.cc:78]  120 in 0.002s
I0216 06:37:15.662400 26802 database_cache.cc:94] Loading images...
I0216 06:37:15.669781 26802 database_cache.cc:143]  16 in 0.007s (connected 16)
I0216 06:37:15.669790 26802 database_cache.cc:154] Building correspondence graph...
W0216 06:37:15.671166 26802 correspondence_graph.cc:156] Duplicate correspondence between point2D_idx=1233 in image_id=1 and point2D_idx=154 in image_id=2
W0216 06:37:15.671173 26802 correspondence_graph.cc:156] Duplicate correspondence between point2D_idx=5237 in image_id=1 and point2D_idx=648 in image_id=2
W0216 06:37:15.671175 26802 correspondence_graph.cc:156] Duplicate correspondence between point2D_idx=6238 in image_id=1 and point2D

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.244762e+03    0.00e+00    3.71e+05   0.00e+00   0.00e+00  1.00e+04        0    6.07e-03    9.32e-03
   1  9.630585e+02    2.82e+02    1.65e+04   2.78e+01   9.96e-01  3.00e+04        1    8.61e-03    1.80e-02
   2  9.480403e+02    1.50e+01    9.61e+03   5.01e+01   9.64e-01  9.00e+04        1    8.43e-03    2.64e-02
   3  9.430516e+02    4.99e+00    7.61e+03   4.26e+01   9.92e-01  2.70e+05        1    8.38e-03    3.48e-02
   4  9.428982e+02    1.53e-01    9.85e+03   5.17e+01   1.77e-01  2.13e+05        1    8.37e-03    4.31e-02
   5  9.420476e+02    8.51e-01    4.28e+03   3.32e+01   8.45e-01  3.17e+05        1    8.43e-03    5.16e-02
   6  9.419183e+02    1.29e-01    5.88e+03   3.92e+01   2.97e-01  2.97e+05        1    8.36e-03    5.99e-02
   7  9.415445e+02    3.74e-01    3.32e+03   2.97e+01   7.90e-01  3.69e+05        1    8.44e-03    6.84e-02
   8  9.414018e+02    1.43e-

...

  10  3.670608e+03    3.22e+01    3.38e+04   8.78e+01   6.50e-01  1.08e+05        1    3.48e-02    3.61e-01
  11  3.639670e+03    3.09e+01    3.33e+04   8.60e+01   6.52e-01  1.11e+05        1    3.45e-02    3.95e-01
  12  3.609920e+03    2.98e+01    3.29e+04   8.42e+01   6.53e-01  1.14e+05        1    3.33e-02    4.29e-01
  13  3.581345e+03    2.86e+01    3.24e+04   8.25e+01   6.54e-01  1.18e+05        1    3.26e-02    4.61e-01
  14  3.553927e+03    2.74e+01    3.18e+04   8.08e+01   6.56e-01  1.21e+05        1    3.26e-02    4.94e-01
  15  3.527655e+03    2.63e+01    3.13e+04   7.91e+01   6.57e-01  1.25e+05        1    3.25e-02    5.27e-01
  16  3.502519e+03    2.51e+01    3.07e+04   7.74e+01   6.58e-01  1.29e+05        1    3.23e-02    5.59e-01
  17  3.478510e+03    2.40e+01    3.00e+04   7.56e+01   6.59e-01  1.34e+05        1    3.23e-02    5.91e-01
  18  3.455620e+03    2.29e+01    2.94e+04   7.39e+01   6.60e-01  1.38e+05        1    3.22e-02    6.23e-01
  19  3.433838e+03    2.18e+

I0216 06:37:20.534118 26802 misc.cc:205] 
Bundle adjustment report
------------------------
I0216 06:37:20.534137 26802 bundle_adjustment.cc:942] 
    Residuals : 24032
   Parameters : 14585
   Iterations : 59
         Time : 1.9228 [s]
 Initial cost : 0.408089 [px]
   Final cost : 0.363979 [px]
  Termination : Convergence

I0216 06:37:20.538305 26802 incremental_mapper.cc:175] => Completed observations: 40
I0216 06:37:20.539273 26802 incremental_mapper.cc:178] => Merged observations: 0
I0216 06:37:20.540382 26802 incremental_mapper.cc:160] => Filtered observations: 6
I0216 06:37:20.540386 26802 incremental_mapper.cc:119] => Changed observations: 0.003828
I0216 06:37:20.540391 26802 misc.cc:198] 
Global bundle adjustment


   2  3.193649e+03    2.52e+00    8.28e+02   2.21e+01   9.98e-01  9.00e+04        1    3.27e-02    1.01e-01
   3  3.190961e+03    2.69e+00    2.49e+03   3.74e+01   9.82e-01  2.70e+05        1    3.23e-02    1.33e-01
   4  3.188643e+03    2.32e+00    8.97e+03   5.46e+01   7.80e-01  3.27e+05        1    3.22e-02    1.66e-01
   5  3.186527e+03    2.12e+00    5.91e+03   3.26e+01   7.81e-01  3.98e+05        1    3.22e-02    1.98e-01
   6  3.184850e+03    1.68e+00    6.16e+03   1.57e+01   7.37e-01  4.46e+05        1    3.22e-02    2.30e-01
   7  3.183370e+03    1.48e+00    5.00e+03   4.64e+00   7.84e-01  5.46e+05        1    3.22e-02    2.62e-01
   8  3.182268e+03    1.10e+00    4.64e+03   8.21e+00   7.61e-01  6.36e+05        1    3.22e-02    2.95e-01
   9  3.181383e+03    8.85e-01    3.78e+03   1.26e+01   8.02e-01  8.15e+05        1    3.22e-02    3.27e-01
  10  3.180780e+03    6.03e-01    3.36e+03   1.54e+01   7.85e-01  1.00e+06        1    3.19e-02    3.59e-01
  11  3.180358e+03    4.21e-

I0216 06:37:21.168267 26802 misc.cc:205] 
Bundle adjustment report
------------------------
I0216 06:37:21.168284 26802 bundle_adjustment.cc:942] 
    Residuals : 24100
   Parameters : 14579
   Iterations : 19
         Time : 0.620633 [s]
 Initial cost : 0.368347 [px]
   Final cost : 0.363246 [px]
  Termination : Convergence

I0216 06:37:21.172130 26802 incremental_mapper.cc:175] => Completed observations: 2
I0216 06:37:21.173095 26802 incremental_mapper.cc:178] => Merged observations: 0
I0216 06:37:21.174211 26802 incremental_mapper.cc:160] => Filtered observations: 0
I0216 06:37:21.174214 26802 incremental_mapper.cc:119] => Changed observations: 0.000166
I0216 06:37:21.174218 26802 incremental_mapper.cc:167] => Filtered images: 0
I0216 06:37:21.258669 26802 misc.cc:198] 
Registering image #2 (4)
I0216 06:37:21.258679 26802 incremental_mapper.cc:495] => Image sees 2546 / 6248 points
I0216 06:37:21.320104 26802 misc.cc:205] 
Pose refinement report
----------------------
I0216 06:37:21.

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  4.775836e+03    0.00e+00    3.14e+03   0.00e+00   0.00e+00  1.00e+04        0    3.01e-02    4.53e-02
   1  4.750814e+03    2.50e+01    6.42e+01   7.44e+00   1.00e+00  3.00e+04        1    4.68e-02    9.21e-02
   2  4.750265e+03    5.49e-01    2.56e+02   1.81e+01   1.00e+00  9.00e+04        1    4.80e-02    1.40e-01
   3  4.749474e+03    7.91e-01    9.33e+02   3.97e+01   1.00e+00  2.70e+05        1    4.81e-02    1.88e-01
   4  4.748670e+03    8.04e-01    2.04e+03   6.32e+01   9.91e-01  8.10e+05        1    4.66e-02    2.35e-01
   5  4.748229e+03    4.40e-01    2.24e+03   5.46e+01   9.67e-01  2.43e+06        1    4.58e-02    2.81e-01
   6  4.748118e+03    1.12e-01    4.82e+02   2.35e+01   9.87e-01  7.29e+06        1    4.58e-02    3.26e-01
   7  4.748110e+03    7.17e-03    2.45e+01   4.91e+00   1.00e+00  2.19e+07        1    4.54e-02    3.72e-01
   8  4.748110e+03    6.44e-

In [ ]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
# 利用生成好的文件进行brownv2生成